In [1]:
import asyncio
import time
import ipywidgets as widgets 
from ipywidgets import interact, interact_manual, Layout

# Class zorgt ervoor dat een waarde meegegeven kan worden aan een Button
import traitlets
class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

In [2]:
# Aparte functie voor het wachten op verandering (aka input van de ME)

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget.on_click(getvalue)
    return future

In [10]:
button = LoadedButton(description='Start', value=False)

def if_clicked():
    button.value = True
    
async def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget.on_click(getvalue)
    return future

async def start_function():
    print('Waiting')
    print(f'button value = {button.value}')
    task = loop.create_task(wait_for_change(button))
    await task
    print(f'done waiting on task {task}')
    if_clicked()
    print(f'button value = {button.value}')
    return button.value
    
async def main():
    print('get loop')
     
    print('waiting for create loop task')
    loop_task = loop.create_task(start_function())
    await loop_task
    print('DONE WAITING')
    sa = loop_task.result()
    return sa


display(button)    
# x = await asyncio.create_task(main())
# x = await main()
loop = asyncio.get_event_loop() 
loop.create_task(main())
# x = await main()

LoadedButton(description='Start', style=ButtonStyle())

<Task pending name='Task-10' coro=<main() running at <ipython-input-10-23aac998bb9f>:27>>

get loop
waiting for create loop task
Waiting
button value = False
done waiting on task <Task finished name='Task-12' coro=<wait_for_change() done, defined at <ipython-input-10-23aac998bb9f>:6> result=<Future pending>>
button value = True
DONE WAITING


In [13]:
button = LoadedButton(description='Start', value=False)

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget.on_click(getvalue)
    return future

async def start_function():
    print('Waiting')
    print(f'button value = {button.value}')
    task = await wait_for_change(button)
    print(f'done waiting on task {task}')
    if_clicked()
    print(f'button value = {button.value}')

display(button)
    
print('get loop')
loop = asyncio.get_event_loop()
print('waiting for create loop task')
loop_task = loop.create_task(start_function())
await loop_task
print('DONE WAITING')
sa = loop_task.result()

LoadedButton(description='Start', style=ButtonStyle())

get loop
waiting for create loop task
Waiting
button value = False


CancelledError: 

In [42]:
button = LoadedButton(description='Start', value=False)
sa = None

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget.on_click(getvalue)
    return future

async def start_function():
    global sa
    print('Waiting for input')
    await wait_for_change(button)  # Wachter gebeurt o.a. hierdoor 
    sa = 'ja toch'
    print(sa)


loop = asyncio.get_event_loop()
loop.create_task(start_function())

display(button)

LoadedButton(description='Start', style=ButtonStyle())

Waiting for input
ja toch


In [38]:
print(sa)

ja toch
